In [1]:
import json
import itertools
import os
from tokenizers import Encoding
from typing import List
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer, EarlyStoppingCallback
import torch
from collections import defaultdict
from datasets import Dataset
import pandas as pd
import numpy as np
import evaluate
from sklearn.metrics import f1_score
from collections import Counter
from seqeval.metrics import classification_report
import re
from datetime import datetime
import unicodedata

/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def align_tokens_and_annotations_bio(tokenized: Encoding, annotations):
    tokens = tokenized.tokens
    aligned_labels = ["O"] * len(
        tokens
    )  # Make a list to store our labels the same length as our tokens
    for anno in annotations:
        annotation_token_ix_set = (
            set()
        )  # A set that stores the token indices of the annotation
        for char_ix in range(anno["start"], anno["end"]):
            print('char_ix = ', char_ix)
            token_ix = tokenized.char_to_token(char_ix)
            if token_ix is not None:
                annotation_token_ix_set.add(token_ix)
        if len(annotation_token_ix_set) == 1:
            # If there is only one token
            token_ix = annotation_token_ix_set.pop()
            prefix = (
                "B"  # This annotation spans one token so is prefixed with U for unique
            )
            aligned_labels[token_ix] = f"{prefix}-{anno['tag']}"

        else:

            last_token_in_anno_ix = len(annotation_token_ix_set) - 1
            for num, token_ix in enumerate(sorted(annotation_token_ix_set)):
                if num == 0:
                    prefix = "B"
                elif num == last_token_in_anno_ix:
                    prefix = "I"  # Its the last token
                else:
                    prefix = "I"  # We're inside of a multi token annotation
                aligned_labels[token_ix] = f"{prefix}-{anno['tag']}"
    return aligned_labels

class LabelSet:
    def __init__(self, labels: List[str]):
        self.labels_to_id = {}
        self.ids_to_label = {}
        self.labels_to_id["O"] = 0
        self.ids_to_label[0] = "O"
        num = 0  # in case there are no labels
        # Writing BILU will give us incremental ids for the labels
        for _num, (label, s) in enumerate(itertools.product(labels, "BI")):
            num = _num + 1  # skip 0
            l = f"{s}-{label}"
            self.labels_to_id[l] = num
            self.ids_to_label[num] = l


    def get_aligned_label_ids_from_annotations(self, tokenized_text, annotations):
        raw_labels = align_tokens_and_annotations_bio(tokenized_text, annotations)
        return list(map(self.labels_to_id.get, raw_labels))


def tokenize_token_classification(examples, tokenizer):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding='longest', return_tensors='pt')

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = [tokenized_inputs.token_to_word(i, j) for j in range(len(tokenized_inputs['input_ids'][i]))]  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = torch.tensor(labels)
    return tokenized_inputs

def dict_of_lists(lst_of_dicts):
    result = defaultdict(list)
    for d in lst_of_dicts:
        for key, value in d.items():
            result[key].append(value)
    return dict(result)

def list_of_dicts(dict_of_lists):
    # First, we need to check if all lists are of the same length to ensure correct transformation
    if not all(len(lst) == len(next(iter(dict_of_lists.values()))) for lst in dict_of_lists.values()):
        raise ValueError("All lists in the dictionary must have the same length")

    # Get the length of the items in any of the lists
    length = len(next(iter(dict_of_lists.values())))
    
    # Create a list of dictionaries, one for each index in the lists
    result = []
    for i in range(length):
        # Create a dictionary for the current index 'i' across all lists
        new_dict = {key: dict_of_lists[key][i] for key in dict_of_lists}
        result.append(new_dict)
    
    return result

def sub_shift_spans(text, ents = [], mappings = []):
    for mapping in mappings:
        adjustment = 0
        pattern = re.compile(mapping['pattern'])
        for match in re.finditer(pattern, text):
            match_index = match.start() + adjustment
            match_contents = match.group()
            if all(mapping['check'](char) for char in match_contents):
                subbed_text = mapping['target'].replace('placeholder', match_contents)
            else:
                subbed_text = mapping['target']
            len_diff = len(subbed_text) - len(match_contents)
            text = text[:match_index] + subbed_text + text[match_index + len(match_contents):]
            if ents:
                if isinstance(ents, list):
                    for ent in ents:
                        if ent['start'] <= match_index and ent['end'] > match_index:
                            ent['end'] += len_diff
                        if ent['start'] > match_index:
                            ent['start'] += len_diff
                            ent['end'] += len_diff
                elif isinstance(ents, dict):
                    if ents['value']['start'] <= match_index and ents['value']['end'] > match_index:
                        ents['value']['end'] += len_diff
                    if ents['value']['start'] > match_index:
                        ents['value']['start'] += len_diff
                        ents['value']['end'] += len_diff

            adjustment += len_diff

    return text, ents

def span_to_words_annotation(samples, target_tag = '', mappings = {}, labels_model = []):
    samples_new = []
    # if not any([l for l in samples['annotations']]):
        
    for i in range(len(samples['data'])):
        text, annotation_list = samples['data'][i]['text'], samples['annotations'][i][0]['result']
        labels_text = []
        tokens = []
        if not annotation_list:
            annotation_list = [[]]
        for j, annotation in enumerate(annotation_list):
            if isinstance(annotation, dict):
                if annotation['value']['labels'][0] != target_tag:
                    continue
            text_subshifted, ents = sub_shift_spans(text, annotation, mappings=mappings)
            text_subshifted_matches = re.finditer(r'[^\s]+', text_subshifted)
            labels_words = []
            first = True
            for regex_match in text_subshifted_matches:
                if j == 0:
                    tokens.append(regex_match.group())
                if isinstance(annotation, dict):
                    if regex_match.start() < ents['value']['start']:
                        labels_words.append(labels_model.labels_to_id['O'])
                    elif regex_match.start() >= ents['value']['start'] and regex_match.end() <= ents['value']['end']:
                        if first:
                            labels_words.append(labels_model.labels_to_id['B-' + ents['value']['labels'][0]])
                            first = False
                        elif not first:
                            labels_words.append(labels_model.labels_to_id['I-' + ents['value']['labels'][0]])
                    else:
                        labels_words.append(labels_model.labels_to_id['O'])
                    labels_text.append({'labels': labels_words, 'tag': annotation['value']['labels'][0]})
        allowed_labels = [labels_model.labels_to_id['O'],
                          labels_model.labels_to_id['B-' + target_tag],
                          labels_model.labels_to_id['I-' + target_tag],
                          ]
        # if the training sample has no tags that we need, we just produce a 0s list
        if target_tag not in [labels['tag'] for labels in labels_text]:
            labels = [0] * len(tokens)
            tag = 'no_tag'
        # if the training sample has tags we need, we first exclude the label lists whose tags don't match
        # and then we merge the label lists that have tags that match the target tag
        else:
            labels = [max(values) for values in zip(*[labels['labels'] for labels in labels_text if labels['tag'] == target_tag])]
            labels = [(label if label in allowed_labels else 0) for label in labels]
            tag = target_tag
        samples_new.append({
            'id': i,
            'ner_tags': labels,
            'tokens': tokens,
            'tag': tag,
        })
    return samples_new

regex_tokenizer_mappings = [
    {'pattern': r'(?<!\s)([^\w\s])|([^\w\s])(?!\s)',
    'target': ' placeholder ',
    'check': lambda x: unicodedata.category(x).startswith('P'),
    },
    {'pattern': r'\s+',
     'target': ' ',
     'check': lambda x: False if re.match('\s+', x) is None else True,
     },
    ]

def compute_metrics_wrapper(label_list, pt, model_name_simple, date_time):
    def compute_metrics(eval_preds):
        nonlocal label_list
        nonlocal pt
        logits, labels = eval_preds
        predictions = np.argmax(logits, axis=2)

        # Extract the true predictions and labels from the sequences
        true_predictions = [
            [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]

        # Compute sequence-level evaluation metrics
        results = classification_report(true_predictions, true_labels, output_dict=True)

        # Flatten the lists to calculate micro F1-score and supports
        flat_true_predictions = [item for sublist in true_predictions for item in sublist]
        flat_true_labels = [item for sublist in true_labels for item in sublist]

        # Calculate micro F1-score using sklearn
        micro_f1 = f1_score(flat_true_labels, flat_true_predictions, average='micro')

        # Prepare the results dictionary
        flat_results = {'micro_f1': float(micro_f1)}

        # Add detailed metrics for each label to the results dictionary
        for label, metrics in results.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    flat_results[f'{label}_{metric}'] = float(value)

        # Compute support for each label using Counter
        label_support = Counter(flat_true_labels)
        for label, count in label_support.items():
            flat_results[f'{label}_support'] = count
        
        models_dir = '/home/lgiordano/LUCA/checkthat_GITHUB/models/M2'
        model_save_name = f'{model_name_simple}_{tt[0]}_target={tt[1]}_SUBSAMPLED_{date_time}'
        model_save_dir = os.path.join(models_dir, date_time, model_save_name)
        if not os.path.exists(model_save_dir):
            os.makedirs(model_save_dir)

        with open(os.path.join(model_save_dir, 'results.json'), 'w', encoding='utf8') as f:
            json.dump(flat_results, f, ensure_ascii = False)

        return flat_results
    return compute_metrics

In [3]:
date_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

train_data_path = '/home/lgiordano/LUCA/checkthat_GITHUB/data/formatted/train_sentences.json'
with open(train_data_path, 'r', encoding='utf8') as f:
    dataset_raw = json.load(f)

df_raw = pd.DataFrame(dataset_raw)  

df_pos = df_raw[df_raw['annotations'].apply(lambda x: len(x[0]['result']) > 0)]
df_neg = df_raw[df_raw['annotations'].apply(lambda x: x[0]['result'] == [])].sample(len(df_pos))
df = pd.concat([df_pos,df_neg])

target_tags = ["Appeal_to_Authority", "Appeal_to_Popularity","Appeal_to_Values","Appeal_to_Fear-Prejudice","Flag_Waving","Causal_Oversimplification",
               "False_Dilemma-No_Choice","Consequential_Oversimplification","Straw_Man","Red_Herring","Whataboutism","Slogans","Appeal_to_Time",
               "Conversation_Killer","Loaded_Language","Repetition","Exaggeration-Minimisation","Obfuscation-Vagueness-Confusion","Name_Calling-Labeling",
               "Doubt","Guilt_by_Association","Appeal_to_Hypocrisy","Questioning_the_Reputation"]
target_tags = [(i, el.strip()) for i, el in enumerate(target_tags)]


In [10]:
shift = 0
for i, tt in enumerate(target_tags[12:]):
    if i < shift:
        continue
    print(f'Training model no. {i} of {len(target_tags)} for {tt[1]} persuasion technique...')
    labels_model = LabelSet(labels=[tt[1]])
    
    df_list = df.to_dict(orient='records')
    df_list_binary = span_to_words_annotation(dict_of_lists(df_list), target_tag=tt[1], mappings=regex_tokenizer_mappings, labels_model=labels_model)
    df_binary = pd.DataFrame(df_list_binary)
    df_binary_pos = df_binary[df_binary['tag'] == tt[1]]
    df_binary_neg = df_binary[df_binary['tag'] != tt[1]].sample(len(df_binary_pos))
    df_binary_subsampled = pd.concat([df_binary_pos, df_binary_neg])#.sample(1000)

    binary_dataset = Dataset.from_pandas(df_binary_subsampled[['id', 'ner_tags', 'tokens']])

    split_ratio = 0.2
    split_seed = 42
    datadict = binary_dataset.train_test_split(split_ratio, seed=split_seed)

    #model_name = 'bert-base-multilingual-cased'
    #model_name = 'xlm-roberta-base'
    model_name = 'microsoft/mdeberta-v3-base'
    model_name_simple = model_name.split('/')[-1]
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    batch_size = 16
    datadict = datadict.map(lambda x: tokenize_token_classification(x, tokenizer), batched=True, batch_size=None)

    columns = [
                'input_ids',
                'token_type_ids',
                'attention_mask',
                'labels'
                ]

    datadict.set_format('torch', columns = columns)

    train_data = datadict['train']
    val_data = datadict['test']

    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding='longest')

    model = AutoModelForTokenClassification.from_pretrained(model_name,
                                                                num_labels=len(labels_model.ids_to_label.values()),
                                                                label2id=labels_model.labels_to_id,
                                                                id2label=labels_model.ids_to_label,
                                                                )
    
    training_args = TrainingArguments(output_dir='/home/lgiordano/LUCA/checkthat_GITHUB/models/M2/mdeberta-v3-base-NEW',
                                  save_total_limit=2,
                                  save_strategy='epoch',
                                  load_best_model_at_end=True,
                                  save_only_model=True,
                                  metric_for_best_model='eval_macro avg_f1-score',
                                  logging_strategy='epoch',
                                  evaluation_strategy='epoch',
                                  learning_rate=5e-5,
                                  optim='adamw_torch',
                                  num_train_epochs=10)
    
    early_stopping = EarlyStoppingCallback(early_stopping_patience=2)

    trainer = Trainer(model,
                      training_args,
                      train_dataset=train_data,
                      eval_dataset=val_data,
                      data_collator=data_collator,
                      tokenizer=tokenizer,
                      compute_metrics=compute_metrics_wrapper(
                          label_list=[i for i in labels_model.ids_to_label.values()],
                          pt=tt[1],
                          model_name_simple=model_name_simple,
                          date_time=date_time),
                      callbacks=[early_stopping])
    
    trainer.train()

    trainer.evaluate()

Training model no. 0 of 23 for Appeal_to_Time persuasion technique...


/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Map: 100%|██████████| 65/65 [00:00<00:00, 4203.94 examples/s]
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/mdeberta-v

Epoch,Training Loss,Validation Loss,Micro F1,Appeal To Time Precision,Appeal To Time Recall,Appeal To Time F1-score,Appeal To Time Support,Micro avg Precision,Micro avg Recall,Micro avg F1-score,Micro avg Support,Macro avg Precision,Macro avg Recall,Macro avg F1-score,Macro avg Support,Weighted avg Precision,Weighted avg Recall,Weighted avg F1-score,Weighted avg Support,O Support,B-appeal To Time Support,I-appeal To Time Support
1,0.729400,0.602196,0.729904,0.037037,0.022727,0.028169,44.000000,0.037037,0.022727,0.028169,44.000000,0.037037,0.022727,0.028169,44.000000,0.037037,0.022727,0.028169,44.000000,633,27,273
2,0.437200,0.655504,0.752412,0.148148,0.086957,0.109589,46.000000,0.148148,0.086957,0.109589,46.000000,0.148148,0.086957,0.109589,46.000000,0.148148,0.086957,0.109589,46.000000,633,27,273
3,0.256300,0.743638,0.743837,0.222222,0.136364,0.169014,44.000000,0.222222,0.136364,0.169014,44.000000,0.222222,0.136364,0.169014,44.000000,0.222222,0.136364,0.169014,44.000000,633,27,273
4,0.144700,0.921970,0.744909,0.296296,0.190476,0.231884,42.000000,0.296296,0.190476,0.231884,42.000000,0.296296,0.190476,0.231884,42.000000,0.296296,0.190476,0.231884,42.000000,633,27,273
5,0.072600,1.180071,0.744909,0.259259,0.166667,0.202899,42.000000,0.259259,0.166667,0.202899,42.000000,0.259259,0.166667,0.202899,42.000000,0.259259,0.166667,0.202899,42.000000,633,27,273
6,0.049600,1.071147,0.748124,0.185185,0.116279,0.142857,43.000000,0.185185,0.116279,0.142857,43.000000,0.185185,0.116279,0.142857,43.000000,0.185185,0.116279,0.142857,43.000000,633,27,273


Training model no. 1 of 23 for Conversation_Killer persuasion technique...


/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Map: 100%|██████████| 389/389 [00:00<00:00, 4250.02 examples/s]
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/mdeberta

...

/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Map: 100%|██████████| 340/340 [00:00<00:00, 5570.56 examples/s]
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/mdeberta

Epoch,Training Loss,Validation Loss,Micro F1,Appeal To Hypocrisy Precision,Appeal To Hypocrisy Recall,Appeal To Hypocrisy F1-score,Appeal To Hypocrisy Support,Micro avg Precision,Micro avg Recall,Micro avg F1-score,Micro avg Support,Macro avg Precision,Macro avg Recall,Macro avg F1-score,Macro avg Support,Weighted avg Precision,Weighted avg Recall,Weighted avg F1-score,Weighted avg Support,O Support,B-appeal To Hypocrisy Support,I-appeal To Hypocrisy Support
1,0.675600,0.600889,0.709728,0.073529,0.032362,0.044944,309.000000,0.073529,0.032362,0.044944,309.000000,0.073529,0.032362,0.044944,309.000000,0.073529,0.032362,0.044944,309.000000,2721,136,2879
2,0.461300,0.586086,0.748431,0.323529,0.166667,0.220000,264.000000,0.323529,0.166667,0.220000,264.000000,0.323529,0.166667,0.220000,264.000000,0.323529,0.166667,0.220000,264.000000,2721,136,2879
3,0.294100,0.762761,0.754184,0.330882,0.194805,0.245232,231.000000,0.330882,0.194805,0.245232,231.000000,0.330882,0.194805,0.245232,231.000000,0.330882,0.194805,0.245232,231.000000,2721,136,2879
4,0.166900,0.975027,0.751220,0.227941,0.158163,0.186747,196.000000,0.227941,0.158163,0.186747,196.000000,0.227941,0.158163,0.186747,196.000000,0.227941,0.158163,0.186747,196.000000,2721,136,2879
5,0.108400,0.932131,0.768131,0.367647,0.264550,0.307692,189.000000,0.367647,0.264550,0.307692,189.000000,0.367647,0.264550,0.307692,189.000000,0.367647,0.264550,0.307692,189.000000,2721,136,2879
6,0.056800,1.181026,0.752266,0.272059,0.185000,0.220238,200.000000,0.272059,0.185000,0.220238,200.000000,0.272059,0.185000,0.220238,200.000000,0.272059,0.185000,0.220238,200.000000,2721,136,2879
7,0.035700,1.306478,0.776848,0.411765,0.321839,0.361290,174.000000,0.411765,0.321839,0.361290,174.000000,0.411765,0.321839,0.361290,174.000000,0.411765,0.321839,0.361290,174.000000,2721,136,2879
8,0.030200,1.370492,0.767434,0.345588,0.258242,0.295597,182.000000,0.345588,0.258242,0.295597,182.000000,0.345588,0.258242,0.295597,182.000000,0.345588,0.258242,0.295597,182.000000,2721,136,2879
9,0.012800,1.442565,0.771792,0.338235,0.234694,0.277108,196.000000,0.338235,0.234694,0.277108,196.000000,0.338235,0.234694,0.277108,196.000000,0.338235,0.234694,0.277108,196.000000,2721,136,2879


Training model no. 10 of 23 for Questioning_the_Reputation persuasion technique...


/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Map: 100%|██████████| 874/874 [00:00<00:00, 1835.45 examples/s]
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/mdeberta

Epoch,Training Loss,Validation Loss,Micro F1,Questioning The Reputation Precision,Questioning The Reputation Recall,Questioning The Reputation F1-score,Questioning The Reputation Support,Micro avg Precision,Micro avg Recall,Micro avg F1-score,Micro avg Support,Macro avg Precision,Macro avg Recall,Macro avg F1-score,Macro avg Support,Weighted avg Precision,Weighted avg Recall,Weighted avg F1-score,Weighted avg Support,O Support,B-questioning The Reputation Support,I-questioning The Reputation Support
1,0.610800,0.537043,0.757313,0.195652,0.124506,0.152174,506.000000,0.195652,0.124506,0.152174,506.000000,0.195652,0.124506,0.152174,506.000000,0.195652,0.124506,0.152174,506.000000,7618,322,5221
2,0.456800,0.530318,0.767267,0.316770,0.190299,0.237762,536.000000,0.316770,0.190299,0.237762,536.000000,0.316770,0.190299,0.237762,536.000000,0.316770,0.190299,0.237762,536.000000,7618,322,5221
3,0.295900,0.726654,0.767495,0.220497,0.159193,0.184896,446.000000,0.220497,0.159193,0.184896,446.000000,0.220497,0.159193,0.184896,446.000000,0.220497,0.159193,0.184896,446.000000,7618,322,5221
4,0.183000,0.890534,0.764608,0.208075,0.146608,0.172015,457.000000,0.208075,0.146608,0.172015,457.000000,0.208075,0.146608,0.172015,457.000000,0.208075,0.146608,0.172015,457.000000,7618,322,5221


In [ ]:
### MACRO F1 (across PTs) = 0.263